Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [16]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [17]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [21]:
train, val = train_test_split(train, train_size=0.80, test_size=0.20,
                              stratify=train['status_group'],random_state=42)

print(train.shape)
print(val.shape)
print(test.shape)

(47520, 41)
(11880, 41)
(14358, 40)


In [22]:
def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id']
    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X['years'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['years'].isnull()
    
    # return the wrangled dataframe
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [41]:
import category_encoders as ce
import numpy as np
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint, uniform

In [27]:
target = 'status_group'

train_features = train.drop(columns=[target])
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

cardinality = train_features.select_dtypes(exclude='number').nunique()
categorical_features = cardinality[cardinality <= 50].index.tolist()
features = numeric_features + categorical_features
x_train = train[features]
y_train = train[target]
x_val = val[features]
y_val = val[target]
x_test = test[features]

In [56]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=42)
)

param_distributions = { 
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=100, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(x_train, y_train);

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed: 1

In [57]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation', search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.33218885347401983, 'randomforestclassifier__n_estimators': 482, 'simpleimputer__strategy': 'mean'}
Cross-validation 0.8019570707070707


In [59]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,80,52,17,76,60,41,82,81,15,9,...,18,85,66,92,58,16,88,93,86,70
mean_fit_time,80.7124,39.1316,27.8238,64.772,66.0071,51.3073,13.8557,8.17626,22.3108,64.2946,...,12.3995,29.1668,66.2586,33.0878,18.0199,23.9335,62.1045,36.3281,3.56825,3.37641
std_fit_time,0.263843,0.502884,0.377659,1.79785,1.84013,0.936838,0.300281,0.235265,0.439871,1.72541,...,0.401917,0.527592,1.62571,0.561517,0.0991838,0.0952193,0.985509,0.784743,0.0771977,0.0591167
mean_score_time,3.13652,2.79621,3.07095,3.31402,3.02308,2.31344,1.19142,0.835905,2.85677,1.83982,...,0.484107,0.741675,1.2096,0.719321,0.48161,0.533818,0.987565,0.686291,0.633909,0.702973
std_score_time,0.172598,0.13732,0.146808,0.0398581,0.0605328,0.0151875,0.00403122,0.00865205,0.0386109,0.0488575,...,0.0192757,0.0135023,0.0241116,0.0421036,0.0234583,0.0185879,0.0230328,0.0085842,0.0488051,0.00340298
param_randomforestclassifier__max_depth,20,20,20,20,20,20,20,20,20,20,...,5,5,5,5,5,5,5,5,5,5
param_randomforestclassifier__max_features,0.332189,0.176223,0.0945768,0.257952,0.286953,0.284388,0.146744,0.112559,0.0689288,0.448569,...,0.547721,0.669994,0.836657,0.841209,0.792531,0.962936,0.968543,0.960879,0.0578002,0.0229237
param_randomforestclassifier__n_estimators,482,402,431,491,469,357,157,99,421,266,...,118,253,485,245,132,140,407,236,179,228
param_simpleimputer__strategy,mean,median,mean,median,median,median,mean,median,median,median,...,mean,median,mean,mean,median,median,mean,mean,mean,mean
params,"{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...","{'randomforestclassifier__max_depth': 20, 'ran...",...,"{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand...","{'randomforestclassifier__max_depth': 5, 'rand..."
split0_test_score,0.801351,0.801477,0.800341,0.800341,0.800909,0.800088,0.800846,0.800467,0.798573,0.799457,...,0.716955,0.71929,0.720048,0.719606,0.718912,0.718344,0.715061,0.714998,0.709065,0.67138


In [60]:
pipeline2 = search.best_estimator_

In [63]:
#if we were going to use how best estimators to predict this is what
#what we would do

x_test = test[features]
y_test = test[target]

y_pred = pipeline2.predict(x_test)


KeyError: 'status_group'

In [64]:
mae = mean_absolute_error(y_test, y_pred)
print(f'Test MAE: ${mae:,.0f}')

NameError: name 'y_test' is not defined

In [70]:
test[features]

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,year_recorded,...,source_type,source_class,waterpoint_type,waterpoint_type_group,longitude_MISSING,latitude_MISSING,construction_year_MISSING,gps_height_MISSING,population_MISSING,years_MISSING
0,0.0,1996.0,35.290799,-4.059696,0,21,3,321.0,2012.0,2013,...,rainwater harvesting,surface,other,other,False,False,False,False,False,False
1,0.0,1569.0,36.656709,-3.309214,0,2,2,300.0,2000.0,2013,...,spring,groundwater,communal standpipe,communal standpipe,False,False,False,False,False,False
2,0.0,1567.0,34.767863,-5.004344,0,13,2,500.0,2010.0,2013,...,rainwater harvesting,surface,other,other,False,False,False,False,False,False
3,0.0,267.0,38.058046,-9.418672,0,80,43,250.0,1987.0,2013,...,shallow well,groundwater,other,other,False,False,False,False,False,False
4,500.0,1260.0,35.006123,-10.950412,0,10,3,60.0,2000.0,2013,...,spring,groundwater,communal standpipe,communal standpipe,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14353,0.0,34.0,38.852669,-6.582841,0,6,1,20.0,1988.0,2011,...,river/lake,surface,communal standpipe,communal standpipe,False,False,False,False,False,False
14354,1000.0,NaN,37.451633,-5.350428,0,4,7,2960.0,1994.0,2011,...,shallow well,groundwater,hand pump,hand pump,False,False,False,True,False,False
14355,0.0,1476.0,34.739804,-4.585587,0,13,2,200.0,2010.0,2013,...,dam,surface,communal standpipe,communal standpipe,False,False,False,False,False,False
14356,0.0,998.0,35.432732,-10.584159,0,10,2,150.0,2009.0,2013,...,river/lake,surface,communal standpipe,communal standpipe,False,False,False,False,False,False
